In [2]:
# the two models we wish to compare
model_path_1 = "../../../MedCAT/temp/model_packs/20230227__kch_gstt_trained_model_494c3717f637bb89.zip"
# 30% trained on MIMIC-IV discharge notes
model_path_2 = "../../data/medcat_models/modelpack/SNOMED2024_UK_final_MIMIC_IV_0_3_71e9e367f38c3b25.zip"
# the documents file we'll be looking at
documents_file = "data/some_synthetic_data.csv"

In [3]:
from compare import get_diffs_for
from output import parse_and_show, show_dict_deep

cdb_comp, tally1, tally2, ann_diffs = get_diffs_for(model_path_1, model_path_2, documents_file)

/Users/martratas/Documents/CogStack/.MedCAT.nosync/working_with_cogstack/venv310/lib/python3.10/site-packages/medcat/cat.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Loading [1] ../../../MedCAT/temp/model_packs/20230227__kch_gstt_trained_model_494c3717f637bb89.zip


/Users/martratas/Documents/CogStack/.MedCAT.nosync/working_with_cogstack/venv310/lib/python3.10/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Loading [2] ../../data/medcat_models/modelpack/SNOMED2024_UK_final_MIMIC_IV_0_3_71e9e367f38c3b25.zip
Counting [1]


100%|██████████| 60/60 [00:08<00:00,  7.15it/s]


...

100%|██████████| 60/60 [00:01<00:00, 34.07it/s]


CDB compare


keys: 100%|██████████| 794151/794151 [00:02<00:00, 319235.36it/s]


Per annotations diff finding


100%|██████████| 60/60 [00:09<00:00,  6.08it/s]


In [4]:
# show results
parse_and_show(cdb_comp, tally1, tally2, ann_diffs)

CDB overall differences:
names.keys.joint                        	752042                                  	                                        
names.keys.total                        	760283                                  	785910                                  
names.keys.not_in_                      	33868                                   	8241                                    
names.values.joint                      	2327941                                 	                                        
names.values.unique_in_                 	752906                                  	152108                                  
names.values.not_in_                    	170834                                  	810321                                  
names.values.total                      	3149859                                 	2510372                                 
snames.keys.joint                       	752042                                  	                                

In [4]:
# you can play with individual parts as well.
# for example, isolate a specific document
ann_diffs.per_doc_results.keys()

for key in ann_diffs.per_doc_results:
    print('='*20,f'\n{key}', f'\n{"="*20}')
    show_dict_deep(ann_diffs.per_doc_results[key].nr_of_comparisons)

doc_0 
IDENTICAL                               	40                                      	                                        
FIRST_HAS                               	6                                       	                                        
SECOND_HAS                              	5                                       	                                        
SAME_SPAN_DIFF_CONCEPT                  	5                                       	                                        
OVERLAPP_1ST_LARGER_DIFF_CONCEPT        	4                                       	                                        
doc_1 
IDENTICAL                               	27                                      	                                        
FIRST_HAS                               	10                                      	                                        
SECOND_HAS                              	4                                       	                                        
SA